In [15]:
import numpy as np
import itertools
import collections
import os,sys
import scipy.constants

In [21]:
scipy.constants.gas_constant

8.3144598

In [2]:
with open('TutA_export.vox') as f:
    data = [line.split() for line in f]
    info = dict((var.strip(), float(num.strip())) for var, num in data[0:9])
    if data[9] == ['nodata_value', 'out']:
        print("Warning: nodata_value in line 9.")
        print("There are cells above the surface.")
        print("See TOP_BC_{} for boundary conditions.".format(f.name[:-4]))
        units = list(itertools.chain(*data[10:]))
    else:
        units = list(itertools.chein(*data[9:]))     

#collections.OrderedDict(sorted(info.items(), key=lambda t: t[0]))

There are cells above the surface.
See TOP_BC_TutA_export for boundary conditions.


In [48]:
try:
    fT = open("TOP_BC_{}.temp".format(f.name[:-4]),'w')
    fh = open("TOP_BC_{}.head".format(f.name[:-4]),'w')
    fp = open("TOP_BC_{}.pres".format(f.name[:-4]),'w')
except IOError:
    print("Can not open file TOP_BC_{}.temp for writing".format(f.name[:-4]))
    print("Can not open file TOP_BC_{}.head for writing".format(f.name[:-4]))
    print("Can not open file TOP_BC_{}.pres for writing".format(f.name[:-4]))
    

In [37]:
p0 = 0.101325
lapserate = -0.0065
t_surf = 288.15
pres = p0 * (1 + lapserate/t_surf * (iz*info['dz'] + info['z0']))**((-scipy.constants.g*0.0289644)/(lapserate*scipy.constants.R))

In [57]:
try:
    fT = open("TOP_BC_{}.temp".format(f.name[:-4]),'w')
    fh = open("TOP_BC_{}.head".format(f.name[:-4]),'w')
    fp = open("TOP_BC_{}.pres".format(f.name[:-4]),'w')
except IOError:
    print("Can not open file TOP_BC_{}.temp for writing".format(f.name[:-4]))
    print("Can not open file TOP_BC_{}.head for writing".format(f.name[:-4]))
    print("Can not open file TOP_BC_{}.pres for writing".format(f.name[:-4]))
    



nxyz = int(info['nx']*info['ny']*info['nz'])
unui = {}
uindex = 1
count = 1
form = units[0]
uindex_field = []
s = ""
ix = 1
iy = 1
iz = 1
p0 = .101325
lapserate = -0.0065
t_surf = 286.15

for i in range(1,nxyz):
    if units[i] not in unui: # if the unit is not in the dictionary, add it
        unui.update({units[i]:uindex})
        uname = units[i]
        uindex += 1 # increase the uindex for shemat
    
    if units[i] == units[i-1]: # count up the units found in the voxel file
        count+=1
    
    else:
        #print("{}*{}".format(count,unui[units[i-1]]))
        # append the units and write it in the format x*y until a new
        # unit is reached
        uindex_field.append("{}*{}".format(count,unui[units[i-1]]))
        s += "{}*{} ".format(count,unui[units[i-1]])
        count = 1
    if i == (nxyz-1):
        #print("{}*{}".format(count,unui[units[i]]))
        # don't forget to include the last unit
        uindex_field.append("{}*{}".format(count,unui[units[i]]))
        s += "{}*{}".format(count,unui[units[i]])
    
    
    if ix < info['nx']:
        ix += 1
    elif iy < info['ny']:
        ix = 1
        iy += 1
    else:
        ix = 1
        iy = 1
        iz += 1
        
    if (units[i] == 'out' and iz < info['nz']) or iz == info['nz']:
        fT.write("{} {} {} {} 0 \n".format(ix,iy,iz, ((iz*info['dz'] + info['z0'])*lapserate + t_surf-273.15)))
        fh.write("{} {} {} {} 0 \n".format(ix,iy,iz, head))
        fp.write("{} {} {} {} 0 \n".format(ix,iy,iz, pres))
    else:
        head = iz*info['dz']
        pres = p0 * (1 + lapserate/t_surf * (iz*info['dz'] + info['z0']))**((-scipy.constants.g*0.0289644)/(lapserate*scipy.constants.R))
        

un_counts = collections.Counter(units)
fT.close()
fh.close()
fp.close()

In [60]:
un_counts = collections.Counter(units)
print(un_counts)
print(sum(un_counts.values()))
print(nxyz)

Counter({'LowerCover': 40871, 'Basement': 23727, 'MiddleCover': 19420, 'UpperCover': 17365, 'out': 16558, 'Granite': 7059})
125000
125000


In [33]:
info

{'dx': 200.0,
 'dy': 200.0,
 'dz': 200.0,
 'nx': 50.0,
 'ny': 50.0,
 'nz': 50.0,
 'x0': 100100.0,
 'y0': 2000100.0,
 'z0': -7900.0}

In [58]:
p0 * (1 + lapserate/t_surf * (iz*info['dz'] + info['z0']))**((-scipy.constants.g*0.0289644)/(lapserate*scipy.constants.R))

0.07837015858284371

In [59]:
iz*info['dz'] + info['z0']

2100.0